# 批量归一化
损失出现在最后，后面的层训练较快
数据在最底部
底部的层训练较慢
底部层一变化，所有都得跟着变
最后的那些层需要重新学习多次
·导致收敛变慢
我们可以在学习底部层的时候避免变化顶部层吗?

# 批量归一化
固定小批量里面的均值和方差
$$\mu_B=\frac{1}{|B|}\sum_{i\in B}x_i\mathrm{~and~}\sigma_B^2=\frac{1}{|B|}\sum_{i\in B}(x_i-\mu_B)^2+\epsilon $$
然后再做额外的调整(可学习的参数)
$$x_{i+1}=\gamma\frac{x_i-\mu_B}{\sigma_B}+\beta $$


In [4]:
import math

a=3700/(math.pi*2.25)
a

523.442923946678